In [1]:
import queue
import threading
import pyrealsense2
from realsense_camera import *
import cv2
import time
from ObjectDetector import ObjectDetector
from ObjectDetector import Robot_mm
from kuka import Kuka
from openshowvar import *
import numpy as np
from ultralytics import YOLO
import supervision as sv
import numpy as np

print('go')
robot = openshowvar(ip = '192.168.17.2', port = 7000)
kuka = Kuka(robot)

kuka.read_cartesian()

kuka.set_base(8)
kuka.set_tool(10)

# Set speed (%)
kuka.set_speed(100)
print('ready')

model = YOLO("yolov8n.pt")

# Function to clear the queue
def clearQueue(dataQueue):
    while not dataQueue.empty():
        dataQueue.get()

# Function for the child thread to process coordinates
def childProcess(stopEvent, dataQueue):
    while not stopEvent.is_set():
        try:
            coord = dataQueue.get(timeout=1)     
        except queue.Empty:
            continue
        while True:
            if coord!=None:
                print("coord", coord)
                Robot.Kuka_move(kuka, depth, coord['x_sort'][0])
                time.sleep(2)
                Robot.Kuka_base(kuka) 
                clearQueue(dataQueue)
                break
            else:
                #Robot.Kuka_base(kuka) 
                clearQueue(dataQueue)
                break
            break
        clearQueue(dataQueue)
        

if __name__ == "__main__":
    # Open the video capture device
    #cap = cv2.VideoCapture(1)  
    cap = RealsenseCamera()
    # Create an instance of ObjectDetector
    detector = ObjectDetector()
    Robot=Robot_mm()
    # Create an Event object to signal the child thread to stop
    stopEvent = threading.Event()
    # Create a queue to communicate coordinates between threads
    dataQueue = queue.Queue()
    coord = None
    #dataQueue.put(coord)
    # Create the child thread
    childThread = threading.Thread(target=childProcess, args=(stopEvent, dataQueue,))
    # Start the child thread
    Robot.Kuka_base(kuka) 
    childThread.start()
    while True:
        #ret, frame = cap.read()
        success, depth_colormap, frame, depth = cap.get_frame_stream()
        coord = detector.detect(frame, model) 
        if coord['x_sort']!=[] and dataQueue.empty()==True:
            dataQueue.put(coord)  

        # Display the frame
        cv2.imshow("Frame", frame)
               
        # Check for the 'q' key press to stop the program
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            stopEvent.set()
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()

go
ready


Using TensorFlow backend.


coord {'x_sort': [array([     268.93,      319.36])], 'currentClass': 39}
(268, 319) 0
30.0 65.0 -90
coord {'x_sort': [array([     269.05,      319.35])], 'currentClass': 39}
(269, 319) 283
53.87190250776544 27.995024513292798 193
coord {'x_sort': [array([     268.81,      319.31])], 'currentClass': 39}
(268, 319) 283
54.339979027525544 27.995024513292798 193
coord {'x_sort': [array([     279.06,      270.32]), array([     378.92,      320.39])], 'currentClass': 39}
(279, 270) 0
30.0 65.0 -90
coord {'x_sort': [array([     378.64,      320.76])], 'currentClass': 39}
(378, 320) 281
3.0434236075963987 27.791436733856152 191
coord {'x_sort': [array([     378.87,      320.73])], 'currentClass': 39}
(378, 320) 281
3.0434236075963987 27.791436733856152 191
coord {'x_sort': [array([     378.88,      320.75])], 'currentClass': 39}
(378, 320) 281
3.0434236075963987 27.791436733856152 191


KeyboardInterrupt: 